In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from constrerl.annotator import (
    Annotator,
    EnumERLModel,
    StringERLModel,
    convert_to_enum_model,
    convert_to_string_model,
    load_test,
    load_train,
    Article,
)
from llama_cpp import Llama, ChatCompletionRequestMessage, LlamaGrammar
import json
from tqdm import tqdm
import sys

RuntimeError: Failed to load shared library '/home/kanben/cgv/hereditary/gutbrainie-clef/.venv/lib/python3.12/site-packages/llama_cpp/lib/libllama.so': libcudart.so.12: cannot open shared object file: No such file or directory

In [ ]:
import pathlib as pl

In [ ]:
train_data_dir = pl.Path("data/annotations/train")
test_data_dir = pl.Path("data/annotations/dev")
train_data_files = list(train_data_dir.glob("*.json"))
test_data_files = list(test_data_dir.glob("*.json"))

train_data: dict[str, Article] = {}
for file in train_data_files:
    train_data.update(load_train(file))

In [ ]:
len(train_data)

1567

In [11]:
annotator = Annotator()
failed_articles: list[tuple[Article, Exception]] = []


def export_train_date(train_data: dict[str, Article], path: pl.Path):
    train_data_chats = []
    for article in tqdm(
        train_data.values(), desc="Building train data chats " + path.name
    ):
        # try:
        similar_articles = annotator.find_similar_examples(article.metadata)
        full_chat = similar_articles + [article]
        chat_elements = [Annotator.prompt_and_respone(article) for article in full_chat]
        chat: list[ChatCompletionRequestMessage] = []
        for i, chat_element in enumerate(chat_elements):
            for chat_message in chat_element:
                chat.append(chat_message)
        train_data_chats.append(
            {
                "article": article.metadata.title,
                "messages": chat,
            }
        )
    # except Exception as e:
    #     failed_articles.append(
    #         (
    #             article,
    #             e,
    #         )
    #     )

    with open(path, "w") as f:
        json.dump(train_data_chats, f)
    # jsonl dump
    with open(path.with_suffix(".jsonl"), "w") as f:
        for chat in train_data_chats:
            # with system prompt
            del chat["article"]
            chat["messages"] = annotator.example_messages + chat["messages"]
            f.write(json.dumps(chat) + "\n")


for train_data_file in train_data_files + test_data_files:
    collection_name = train_data_file.name.split(".")[0].split("_")[-1]
    export_train_date(
        load_train(train_data_file),
        pl.Path(
            *train_data_file.parts[:-3], f"train_data_chats_{collection_name}.json"
        ),
    )
export_train_date(train_data, pl.Path("data/annotations/train_data_chats.json"))

Building train data chats train_data_chats_bronze.json: 100%|██████████| 749/749 [01:10<00:00, 10.58it/s]
Building train data chats train_data_chats_gold.json: 100%|██████████| 208/208 [00:18<00:00, 11.08it/s]
Building train data chats train_data_chats_platinum.json: 100%|██████████| 111/111 [00:10<00:00, 10.44it/s]
Building train data chats train_data_chats_silver.json: 100%|██████████| 499/499 [00:51<00:00,  9.77it/s]
Building train data chats train_data_chats_dev.json: 100%|██████████| 40/40 [00:03<00:00, 10.73it/s]
Building train data chats train_data_chats.json: 100%|██████████| 1567/1567 [02:31<00:00, 10.36it/s]


In [ ]:
missing_relations_all = {}
for article, err in failed_articles:
    missing_relations: dict[str, dict[str, str]] = {}
    for e in err.errors():
        index = e["loc"][1]
        spo_id = e["loc"][-1]
        if index not in missing_relations:
            missing_relations[index] = {
                "subject_label": None,
                "predicate": None,
                "object_label": None,
            }
        missing_relations[index][spo_id] = e["input"]
    for index, missing_relation in missing_relations.items():
        names = missing_relation.values()
        spo= "->".join(names)
        if spo not in missing_relations_all:
            missing_relations_all[ "->".join(names)] = []
        missing_relations_all[spo].append(article.metadata.title)

In [ ]:

relations_missing = []
for spo, articles in missing_relations_all.items():
    s, p, o = spo.split("->")
    relations_missing.append(   
            {
                "heads": [s],
                "tails": [o],
                "predicate": [p],
            }
    )
print(json.dumps(relations_missing, indent=2))

[]
